# Extract 2D obstacles
Generate polygons for all static obstacles, and merge these with the sidewalk polygons.

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import geopandas as gpd
import ast
import pathlib
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
from tqdm.notebook import tqdm
tqdm.pandas()

from upcp.utils import bgt_utils
from upcp.utils import csv_utils
from upcp.utils import las_utils

from upc_sw.cluster2polygon import Cluster2Polygon
from upc_sw import sw_utils

## Create polygons of static obstacles
In the previous notebook, we performed a change detection algorithm that calculated M3C2 distance for each point in the point cloud. Based on negative and positive threshold values we can filter for the static points in the point cloud. We then cluster these into individual obstacles and create bounding polygons for each.

In [ ]:
pc_data_folder = '../datasets/pointclouds/'
out_folder = '../datasets/obstacles/'
CRS = 'epsg:28992'

# Distance threshold for static obstacles.
m3c2_threshold = 0.2

# Convert 3D Obstacle blobs to 2D polygons using a clustering algorithm.
# Set use_concave=False to use the faster convex hull.
# Change alpha to determine the 'concaveness' of the concave hull, with 0 being convex.
c2p = Cluster2Polygon(min_component_size=100, grid_size=0.2, use_concave=True, alpha=0.5)

In [ ]:
all_tiles = las_utils.get_tilecodes_from_folder(f'{pc_data_folder}m3c2/')

In [ ]:
tile_tqdm = tqdm(all_tiles, unit='tile', smoothing=0)

obstacle_df = gpd.GeoDataFrame(columns=['tilecode', 'type', 'geometry'], geometry='geometry', crs=CRS)

for tilecode in tile_tqdm:
    tile_tqdm.set_postfix_str(tilecode)
    
    # Read point cloud with M3C2 distances
    in_file = f'{pc_data_folder}m3c2/m3c2_{tilecode}.laz'
    points, m3c2_distance = sw_utils.read_las(in_file, extra_val='M3C2_distance', extra_val_dtype='float32')

    # Filter for static points
    mask = np.abs(m3c2_distance) < m3c2_threshold
    
    # Get the polygons
    polygons, types = c2p.get_obstacle_polygons(points[mask])
    data = {'tilecode': [tilecode]*len(polygons),
            'type': types,
            'geometry': polygons}
    obstacle_df = obstacle_df.append(gpd.GeoDataFrame(data, geometry='geometry', crs=CRS))

In [ ]:
# Save the obstacle GeoDataFrame.
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)
obstacle_df.to_file(f'{out_folder}obstacles.shp')

## Merge sidewalk polygons with obstacles
The found obstacles (polygons) in the previous step are merged with the sidewalk polygons as interiors.

In [ ]:
# Scraped sidewalk data for the area (see Notebook 1)
sidewalk_data = '../datasets/bgt/bgt_voetpad.csv'
obstacle_data = '../datasets/obstacles/obstacles.shp'

In [ ]:
# Read the sidewalk data
sidewalk_df = gpd.read_file(sidewalk_data, crs=CRS)
sidewalk_df = sidewalk_df[sidewalk_df['bgt_name']=='voetpad']

# Convert to Polygons and keep only the necessary columns
sidewalk_df['geometry'] = sidewalk_df.progress_apply(lambda row: Polygon(ast.literal_eval(row['polygon'])), axis=1)
sidewalk_df = sidewalk_df[['bgt_name', 'geometry']]

# Save the sidewalk data.
sidewalk_df.to_file(f'{out_folder}sidewalks.shp')

In [ ]:
obstacle_df = gpd.read_file(obstacle_data, crs=CRS)

In [ ]:
def merge_obstacles(row):
    # Subtract all obstacles that intersect the sidewalk polygon.
    sw_poly = row.geometry
    obst_polys = obstacle_df[obstacle_df.intersects(sw_poly)].geometry.values
    # TODO: do something with obstacle type.
    if len(obst_polys) > 1:
        obst_polys = MultiPolygon(unary_union(obst_polys))
    sw_poly = sw_poly - MultiPolygon(obst_polys)
    if type(sw_poly) == MultiPolygon:
        parts = [p for p in sw_poly.geoms if p.area > 1] # TODO: magic number
        if len(parts) > 1:
            # TODO: do something clever here.
            sw_poly = MultiPolygon(parts)
        else:
            sw_poly = parts[0]
    return sw_poly

sw_merged_df = sidewalk_df.copy()
sidewalk_df['geometry'] = sidewalk_df.progress_apply(merge_obstacles, axis=1)

In [ ]:
# Save the merged sidewelk data.
# TODO: this will throw an error if one of the sidewalks is a MultiPolygon
sidewalk_df.to_file(f'{out_folder}sidewalks_merged.shp')